In [94]:
print("Hello")

Hello


In [95]:
from google import genai
from dotenv import load_dotenv

In [96]:
client = genai.Client()

In [97]:
from google.genai import types

# Define the function declaration properly
weather_function = types.FunctionDeclaration(
    name="get_weather",
    description="Fetch real-time weather information for a given city. Use this tool whenever the user asks about weather, temperature, humidity, or climate of a place.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to fetch weather information for."
            }
        },
        "required": ["city"]
    }
)


In [98]:
from google.genai import types


In [99]:
tools = types.Tool(function_declarations=[weather_function])
config = types.GenerateContentConfig(tools=[tools])


In [100]:
load_dotenv()
import os
import requests

In [101]:
API_KEY = os.getenv("OPEN_WEATHER_MAP")
if(API_KEY):
    print("Found")

Found


In [102]:
def get_weather(city: str):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return data

In [103]:
chat = client.chats.create(
    model="gemini-2.5-flash",
    config=config,
)


In [104]:
response = chat.send_message("What is the weather in New York?")

In [105]:
response.candidates[0].content


Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'city': 'New York'
        },
        name='get_weather'
      ),
      thought_signature=b'\n\xf6\x01\x01r\xc8\xda|EPJE\x9f+F\xd3\x9f0\r\xcaI\xce\xc6\xb6\xb7\xbd\xfaT\xc4f\x81\xc4\xdf\x02d\xfc\xa7t\x83\x97\x02<fI1<\x8ef]/2\xe0?v\xd4\xcei\x15l\xa5\xf8e9}\x15\x16<\xa6\xd5\xfdL\xb4\xc6\x080\xac~\x8a\xb3Vg\xde\xc7A_\xce\x19\xea\x948w;\xe7\x99\x14\xc7\xfc...'
    ),
  ],
  role='model'
)

In [106]:
# function_call = response.candidates[0].content.parts[0].function_call
# function_name = function_call.name
# function_args = function_call.args


In [107]:
for _ in response.candidates:
    for part in _.content.parts:
        if part.function_call:
            function_name = part.function_call.name
            function_args = part.function_call.args["city"]

In [108]:
function_name

'get_weather'

In [109]:
function_args

'New York'

In [110]:
if function_name == 'get_weather':
    response = get_weather(function_args)
    print(response)

{'coord': {'lon': -74.006, 'lat': 40.7143}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 1.43, 'feels_like': -1.5, 'temp_min': -1.53, 'temp_max': 2.3, 'pressure': 1016, 'humidity': 71, 'sea_level': 1016, 'grnd_level': 1015}, 'visibility': 10000, 'wind': {'speed': 2.68, 'deg': 284, 'gust': 3.13}, 'clouds': {'all': 0}, 'dt': 1765088848, 'sys': {'type': 2, 'id': 2037026, 'country': 'US', 'sunrise': 1765109212, 'sunset': 1765142919}, 'timezone': -18000, 'id': 5128581, 'name': 'New York', 'cod': 200}


In [111]:
contents = [
    types.Content(
        role="user", parts=[types.Part(text="What is the weather in New York?")]
    )
]

In [114]:
response = chat.send_message(contents)


ValueError: Message must be a valid part type: typing.Union[str, google.genai.types.File, google.genai.types.Part] or typing.Union[str, google.genai.types.File, google.genai.types.FileDict, google.genai.types.Part, google.genai.types.PartDict], got <class 'list'>